In [1]:
import pandas as pd
import numpy as np

import jax.numpy as jnp
import jax.random as jrnd
from jax.example_libraries import stax
import optax

from distributions import HorseshoeLogisticReg
from utils import run_atess, run_neutra

In [2]:
data = pd.read_table('german.data-numeric', header=None, delim_whitespace=True)
### Pre processing data as in NeuTra paper
y = -1 * (data.iloc[:, -1].values - 2)
X = data.iloc[:, :-1].apply(lambda x: -1 + (x - x.min()) * 2 / (x.max() - x.min()), axis=0).values
# X = data.iloc[:, :-1].apply(lambda x: (x - x.mean()) / x.std(), axis=0).values
X = np.concatenate([np.ones((1000, 1)), X], axis=1)
N_OBS, N_PARAM = X.shape

In [3]:
dist = HorseshoeLogisticReg(X, y)
hidden_size, n_layer, n_flow, n_atoms = N_PARAM, 2, 2, 10
n_hidden = [hidden_size] * n_layer
optim = optax.adam(1e-4)
non_linearity = stax.Tanh

vi_iter, n_epochs, batch_size = 1000, 5, 1000
pre_iter = n_epochs * vi_iter
n_warm = n_epochs * batch_size #too much warm up not good for nuts, it semms
n_warm = vi_iter
n_iter, n_chain = 1000, 1

In [4]:
ksam, kcenter, kinit = jrnd.split(jrnd.PRNGKey(0), 3)
# dist.center_model(kcenter, optim, n_atoms, vi_iter)
dist.initialize_model(kinit, n_chain, autocenter=False)

In [ ]:
run_atess(ksam, dist.logprob_fn, dist.init_params, 
    optim, N_PARAM * 2 + 1, n_flow, n_hidden, non_linearity, 
    n_atoms, vi_iter, n_epochs, batch_size, n_iter, n_chain)

/home/cabezasg/.local/lib/python3.8/site-packages/jax/_src/tree_util.py:188: FutureWarning: jax.tree_util.tree_multimap() is deprecated. Please use jax.tree_util.tree_map() instead as a drop-in replacement.
  warnings.warn('jax.tree_util.tree_multimap() is deprecated. Please use jax.tree_util.tree_map() '


In [ ]:
_, mcmc = run_neutra(ksam, dist.y, dist.model, dist.init_params, 
    optim, N_PARAM * 2 + 1, n_flow, n_hidden, non_linearity, 
    n_atoms, pre_iter, n_warm, n_iter, n_chain)